# Real world example: Upload Data and Create Entities

First, let's see a easy login script to save and use the PAT for accesing openBIS and getting our space automatically, to avoid later problems.

In [ ]:
url    = 'https://main.datastore.bam.de/'
pat    = '' # leave empty to read token from file or authenticate with username
userid = '' # leave empty to use the username from your local machine
space  = '' # leave empty to use the users home space

## Connecting
Use a PAT or username and pasword to connect. A PAT can be specified or read from a file (`OPENBIS_PAT.txt` in current folder or defined by the environment variable `OPENBIS_PAT_FILE`).

After execution the object `o` is the openBIS connection to work with.

In [ ]:
from pybis import Openbis
try:
    from os import environ
    pat = pat or open(environ.get('OPENBIS_PAT_FILE', 'OPENBIS_PAT.txt'), 'r').read().strip()
    o = Openbis(url, token=pat)
    userid = o.token.split('-')[1]
except:
    from getpass import getuser, getpass
    o = Openbis(url)
    userid = userid.lower() or getuser()
    password = getpass('Enter password for user {} at {}: '.format(userid, url))
    o.login(userid, password)
server_info = o.get_server_information()
person = o.get_person(userid)
space = space.upper() or person.space

print('Server: {} (openBIS {}, API {})'.format(o.hostname, server_info.openbis_version, server_info.api_version))
print('UserId: {} ({} {}, {})'.format(person.userId, person.firstName, person.lastName, person.email))
print('Space : {}'.format(space))

# Now do something
Like listing all experimental steps we have access to ...

In [ ]:
o.get_objects(type='EXPERIMENTAL_STEP')

## Create and Save a PAT
If you are logged in with a password you can create and save a PAT here.


In [ ]:
sessionName = 'default'
from datetime import datetime
from os import environ
validFrom = datetime.now() # from now,  or datetime(2024, 12, 17, 10, 30, 0, 0)
validTo   = None # max. time, or datetime(2024, 12, 18, 0, 0, 0, 0)
token = o.get_or_create_personal_access_token(sessionName, validFrom, validTo)
with open(environ.get('OPENBIS_PAT_FILE', 'OPENBIS_PAT.txt'), 'w') as pat_file:
    pat_file.write(token.permId)
print(token.permId)

## Scenario

A measurement setup produces new files which should be uploaded to openBIS. In this example we will use the generic, well-known [IRIS data set](https://en.wikipedia.org/wiki/Iris_flower_data_set).
We want to create code to upload a data set and attacht it to an experimental step. 

The script should do the following:

* make sure the project and experiment exist - create if necessary
* read the measurements description from an additional file (measurement.txt)
* create the name/code of the experimental step with this information
* search this step - create if it is not already there, setting description from measurement.txt
* upload the two files (iris.csv, measurement) as a dataset to an experimental step

This example shows the **interactive development process** - step by step from the first line to the complete script.

## Optional: Create dummy data

In [ ]:
project_code = 'IRIS_PROJECT'
collection_code = 'IRIS_EXPERIMENT'
object_code = 'IRIS_STEP'

my_space = o.get_space(space)

try:
    my_project = my_space.get_project(project_code)
except ValueError:
    my_project = o.new_project(space=my_space, code=project_code)
    my_project.save()

my_collection = o.get_collections(space=my_space.code, project=my_project.code, code=collection_code)[0]
if not my_collection:
    my_collection = o.new_collection(project=my_project, code=collection_code, type='DEFAULT_EXPERIMENT')
    my_collection.save()

my_object = my_space.get_objects(code=object_code, project=my_project, collection=my_collection, type='EXPERIMENTAL_STEP')[0]
if not my_object:
    my_object = o.new_object(code=object_code, collection=my_collection, type='EXPERIMENTAL_STEP')
    my_object.save()
# download the data file
import requests
resp = requests.get('https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv')
with open('iris.csv', 'w') as csvfile:
     csvfile.write(resp.text)
with open('measurement.txt', 'w') as txtfile:
     txtfile.write('foo\nbar\nbaz\n')

## Upload a dataset and attach to an experimental step

### Explore types and entities

#### List dataset types

In [ ]:
o.get_dataset_types() # list dataset types to select the desired one

In [ ]:
# select and store dataset type
dataset_type = 'RAW_DATA'
dataset_type

#### List collections (experiments)

In [ ]:
my_space.get_collections() # list collections to check where we want to upload the dataset

In [ ]:
# select and store this IRIS_EXPERIMENT
my_experiment = my_space.get_collection('IRIS_EXPERIMENT') #save selected collection in a variable
my_experiment

#### List objects (samples or experimental steps)

In [ ]:
my_space.get_objects(collection=my_experiment.code) # list objects to check where we want to upload the dataset

In [ ]:
my_step = my_space.get_objects(code='IRIS_STEP', collection=my_experiment.code)[0] #save selected object in a variable
# or: my_step = my_space.get_object('/MMUSTERM/PYBISTUTORIAL/IRIS_STEP')
my_step

### Upload a dataset and attach to the object
The dataset will contain just two files: `iris.csv` and `measurement.txt`.

In [ ]:
my_dataset = o.new_dataset(
    type = dataset_type, # selected type for the dataset
    collection = my_experiment, # selected collection
    object = my_step, # selected object
    files = ['iris.csv', 'measurement.txt'] # iris dataset to upload
)
my_dataset.save()

### Modify description (property) of the experimental step after upload

In [ ]:
# read the content of measuremet.txt
with open('measurement.txt', 'r') as txtfile:
     desc = txtfile.read()
print(desc)
my_step.props['experimental_step.experimental_description'] = desc
my_step.save()

Now we have all the code to upload a dataset to an existing object and alter it's properties.

## Create the experimental step, experiment and project if needed

### Search or create the experimental step/object

For every measurement series a new experimental step should be used, based on the contents of the file `measurements.txt`. So we need to read this file first and use the first word for code of the experimental step.

In [ ]:
with open('measurement.txt', 'r') as txtfile:
     desc = txtfile.read()
my_step_name = 'IRIS_'+desc.split()[0].upper()
my_step_name

In [ ]:
steps = o.get_objects(my_step_name, project=my_project)
if steps:
    my_step = steps[0]
else:
    my_step = o.new_object(
        type = 'EXPERIMENTAL_STEP',
        project = my_project,
        collection = my_collection,
        code = my_step_name
    )
    my_step.save()
my_step # is now an existing or newly created step

## Putting it all together: the complete script

### The Complete Script

Now we combine all of the code above to a cell/script that can be used standalone. loop

In [ ]:
project_code = 'IRIS_PROJECT'
collection_code = 'IRIS_EXPERIMENT'
object_code = 'IRIS_STEP'

my_space = o.get_space(space)

# project and collection
try:
    my_project = my_space.get_project(project_code)
except ValueError:
    my_project = o.new_project(space=my_space, code=project_code)
    my_project.save()

try:
    my_collection = o.get_collections(space=my_space.code, project=my_project.code, code=collection_code)[0]
    print("OLD")
except ValueError:
    my_collection = o.new_collection(project=my_project, code=collection_code, type='DEFAULT_EXPERIMENT')
    my_collection.save()
    print("NEW")

my_collection

In [ ]:
# settings
project_code = 'IRIS_PROJECT'
collection_code = 'IRIS_EXPERIMENT'
collection_type = 'DEFAULT_EXPERIMENT'
object_type = 'EXPERIMENTAL_STEP' 
dataset_type = 'RAW_DATA'

# space
my_space = o.get_space(space) # get the space which will be used

# project and collection
try:
    my_project = my_space.get_project(project_code)
except ValueError:
    my_project = o.new_project(space=my_space, code=project_code)
    my_project.save()

my_collection = o.get_collections(space=my_space.code, project=my_project.code, code=collection_code)[0]
if not my_collection:
    my_collection = o.new_collection(project=my_project, code=collection_code, type='DEFAULT_EXPERIMENT')
    my_collection.save()

# object/step
with open('measurement.txt', 'r') as txtfile:
     desc = txtfile.read()
my_step_name = 'IRIS_'+desc.split()[0].upper()
steps = o.get_objects(my_step_name, project=my_project)
if steps:
    my_step = steps[0]
else:
    my_step = o.new_object(
        type = object_type,
        project = my_project,
        collection = my_collection,
        code = my_step_name,
        props = {'experimental_step.experimental_description' : desc}
    )
    print(my_step)
    my_step.save()

# dataset
my_dataset = o.new_dataset(
    type = dataset_type, # selected type for the dataset
    collection = my_collection, # selected collection
    object = my_step, # selected object
    files = ['iris.csv', 'measurement.txt'] # iris dataset to upload
)
my_dataset.save()
o.logout()